Este código está un poco complicado, si tienen dudas de la implementación, me pueden contactar por WhatsApp (si no se vuelve a morir) o en 0226154@up.edu.mx

In [8]:
from sympy import Symbol, diff, lambdify
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
x1 = Symbol('x1')
x2 = Symbol('x2')

Utilidades

In [2]:
def partial(expr, element, elements=(x1, x2), val=None):
  assert element in elements
  p = diff(expr, element)
  if val is None:
    return p
  else:
    lp = lambdify([elements], p, 'numpy')
    return lp(val)

In [3]:
def gradient(expr, elements=(x1, x2), val=None):
  return np.matrix(
    [
      [partial(expr, element, elements, val)] 
      for element in elements
    ]
  )


In [4]:
def hessian(expr, elements, val=None):
  return np.matrix(
    [
      [
        partial(
            partial(expr, element_row, elements, None), 
            element_column, 
            elements, 
            val
        )
        for element_column in elements
      ]
      for element_row in elements
    ]
)

Newton Raphson

In [5]:
def n_newton_raphson(expr, p, elements=(x1, x2), e1=1e-4, e2=1e-4):
  '''Version para n dimensiones'''
  f = lambdify([elements], expr, 'numpy')
  while True:
    fp = f(p)
    gradf = gradient(expr, elements, p)
    hessf = hessian(expr, elements, p)
    Sm = -np.linalg.inv(hessf) * gradf
    S = np.squeeze(np.asarray(Sm))
    if f(p + S) - fp < e1 and np.linalg.norm(gradf) < e2:
      break
    p += S
  
  return p, f(p)

Con restricciones

In [6]:
def nr_newton(expr, restrictions, p, rs, elements=(x1, x2), e1=1e-8, e2=1e-8):
  '''Version para n dimensiones con restricciones'''
  f = lambdify(elements, expr, 'numpy')
  for restriction, r in zip(restrictions, rs):
    expr += restriction * r
  
  ans, _ = n_newton_raphson(expr, p, elements, e1, e2)
  return ans, f(ans[0], ans[1])

Uso (ejemplo en el que se buscan los valores óptimos para las restricciones)

In [9]:
r1 = -x1**2 + x2 - 4
r2 = -(x1 - 2)**2 + x2 -3
r1l = lambdify((x1, x2), r1, 'numpy')
r2l = lambdify((x1, x2), r2, 'numpy')

arr = []
for i in range(1, 200):
  rv1, rv2 = np.random.uniform(0., 1., 2)
  try:
    varr, fp = nr_newton((x1 - 1)**2 + (x2 - 5)**2, [r1, r2], (8, 0), [rv1, rv2])
    if r1l(varr[0], varr[1]) <= 0 and r2l(varr[0], varr[1]) <= 0:
      arr.append([varr[0], varr[1], fp, rv1, rv2, r1l(varr[0], varr[1]), r2l(varr[0], varr[1])])
  except Exception as e:
    print(e)

df = pd.DataFrame(data=arr, columns=['x1', 'x2', 'f', 'rv1', 'rv2', 'r1(x1,x2)', 'r2(x1,x2)' ])

In [ ]:
df.sort_values(by=['f'])